In [1]:
using DataFrames, Distributions, DataFramesMeta

In [2]:
function gen_data(
    ; 
    n = 500000, 
    mu = [0, 2], 
    sigma = 1, 
    pZ1 = 0.8)
    
    data = DataFrame(
        Obs_ID = 1:n,
        Z = (rand(Uniform(0,1), n) .> pZ1) .+ 1
        )

    @transform!(data, :Y = rand(Normal(0, sigma), n) + mu[:Z])    
    @select!(data, :Obs_ID, :Y)
    @transform!(data, :p_Y_given_Z_1 = pdf.(Normal(mu[1], sigma), :Y))
    @transform!(data, :p_Y_given_Z_2 = pdf.(Normal(mu[2], sigma), :Y))

    return data
  end

gen_data (generic function with 1 method)

In [3]:
data = gen_data(n = 500000, pZ1 = 0.8);
first(data, 6)

,Obs_ID,Y,p_Y_given_Z_1,p_Y_given_Z_2
,Int64,Float64,Float64,Float64
1,1,-2.29579,0.0286023,3.92385e-5
2,2,3.40976,0.00119195,0.147689
3,3,1.21632,0.190396,0.293459
4,4,1.22822,0.187646,0.296187
5,5,0.386348,0.370252,0.108514
6,6,2.14119,0.0403044,0.394985


In [4]:
function fit_model!(
    data; 
    pi_hat_0 = 0.5, 
    tolerance = 0.00001,
    max_iterations = 1000,
    verbose = false
    )

    # pre-allocate a table of results by iteration:
    progress = DataFrame(
        Iteration = 0:max_iterations, 
        pi_hat = Vector{Float64}(undef, max_iterations+1), 
        loglik = Vector{Float64}(undef, max_iterations+1), 
        diff_loglik = Vector{Float64}(undef, max_iterations+1)
        )

    # initial E step, to perform needed calculations for initial likelihood:
    E_step!(data, pi_hat_0) 
    ll = loglik(data)
    progress[1, :] = (0, pi_hat_0, ll, NaN)
    
    last_iter = 0
    for i in 1:max_iterations

        # M step: re-estimate parameters
        pi_hat = M_step(data)

        # E step: re-compute distribution of missing variables, using parameters
        E_step!(data, pi_hat)
        
        # Assess convergence

        ## save the previous log-likelihood so we can test for convergence
        ll_old = ll
        
        ## here's the new log-likelihood
        ll = loglik(data)

        ll_diff = ll - ll_old
        
        progress[i+1,:] = (i, pi_hat, ll, ll_diff)
        
        if(verbose)
            print(progress[i+1,:])
        end
        
        if ll_diff < tolerance
            last_iter = i
            break
        end
    end
    
    return progress[1:(last_iter + 1), :]
end

fit_model! (generic function with 1 method)

In [5]:
function E_step!(data, pi_hat)
    @transform!(data, :pY_Z1 = :p_Y_given_Z_1 .* pi_hat)
    @transform!(data, :pY_Z2 = :p_Y_given_Z_2 .* (1- pi_hat))
    @transform!(data, :pY = :pY_Z1 + :pY_Z2)
    @transform!(data, :pZ1_given_Y = :pY_Z1 ./ :pY)
end

E_step! (generic function with 1 method)

In [6]:
function M_step(data)
    data[:, :pZ1_given_Y] |> mean
end

M_step (generic function with 1 method)

In [7]:
function loglik(data)
    data[:, :pY] .|> log |> sum
end

loglik (generic function with 1 method)

In [8]:
@time progress = fit_model!(data, tolerance = 0.00001);


LoadError: MethodError: no method matching setindex!(::Tuple{DataFrame, DataFrame}, ::Tuple{Int64, Float64, Float64, Float64}, ::Int64, ::Colon)

The first run requires compilation; on subsequent calls it will be faster:

In [ ]:
@time progress = fit_model!(data, tolerance = 0.00001)

  0.151518 seconds (13.08 k allocations: 504.204 MiB, 15.66% gc time)


,Iteration,pi_hat,loglik,diff_loglik
,Int64,Float64,Float64,Float64
1,0,0.5,-8.78014e5,NaN
2,1,0.6655,-8.3736e5,40653.7
3,2,0.737867,-828371.0,8989.06
4,3,0.770351,-8.26289e5,2081.86
5,4,0.785713,-8.25782e5,506.995
6,5,0.793237,-8.25655e5,127.414
7,6,0.796996,-8.25622e5,32.5826
8,7,0.798893,-8.25614e5,8.40918
9,8,0.799856,-8.25612e5,2.18071
